## NOTE - I have made an assumption that the clients won't churn for a small change in discount percentage, i.e.

In [2]:
### Importing all the requried libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as  np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('../input/ibmvinit/IBM - IBM_CAO_data_challenge_DS_2021_.csv')

df.head()

## 1 Across all orders, which pair of product categories appears the most? (For example, SS and HW)


In [4]:
# Method 1 
## getting all the order pairs from the data frame 
from itertools import combinations
cats=df.groupby('ORDER_ID')['PROD_CAT']\
    .agg(list)\
    .apply(lambda x:list(combinations(set(x),2)))\
    .explode()

cats.value_counts()


In [5]:
##Method 2
ct = pd.crosstab(df['ORDER_ID'], df['PROD_CAT']).gt(0).astype(int)
co_occur = (ct.T @ ct)


counts = co_occur.where(np.tri(len(co_occur), k=-1, dtype=bool)).stack()

counts.idxmax()

### (SW,HW) pair appears the most i.e. 5895 times 

## For the pair identified in 1a, in what percent of orders does it appear?						


In [1]:
#df["PROD_CAT"].value_counts()


In [2]:
#df["ORDER_ID"].value_counts()


In [7]:
## finding the percentage of SW and HW order pair in the orders 
order_Count = df["ORDER_ID"].nunique()

most_occuring = 5895

quotient = most_occuring / order_Count

percent = quotient * 100

percent

### (SW,HW) appears 57.23 % times in orders

## How does the frequency of product pairs across orders vary by industry?						


In [8]:
df["PROD_ID"].value_counts()


In [9]:
# Method 1 
from itertools import combinations
cats=df.groupby('INDUSTRY')['PROD_ID']\
    .agg(list)\
    .apply(lambda x:list(combinations(set(x),2)))\
    .explode()

cats


In [10]:
cats=df.groupby(['ORDER_ID','PROD_CAT','INDUSTRY'])
cats
#print(cats.get_group())

### IT industry has highest frequency, followed by Finance and Real Estate industries. Healthcare and retail have lowest frequency of order pairs. 


## 3. What are the most significant drivers for discount?




In [11]:
df['Discount_Percentage'] = ((df['PRICE_ORIG'] - df['PRICE_DISC'])/ df['PRICE_ORIG']) * 100


In [15]:
#df.tail(100)

In [13]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

# label_encoder object knows how to understand word labels.
## converting the Categorical column into numerical columns
 
df['PROD_CAT']=le.fit_transform(df['PROD_CAT'])
df['INDUSTRY']=le.fit_transform(df['INDUSTRY'])
df['SIZE']=le.fit_transform(df['SIZE'])
df['STATE']=le.fit_transform(df['STATE'])
df['ORDER_ID']=le.fit_transform(df['ORDER_ID'])


In [14]:
df['ORDER_ID'].value_counts()

In [15]:
import seaborn as sns
#Using Pearson Correlation
plt.figure(figsize=(25,25))
cor = df.corr()  # plot the heatmap on X
sns.heatmap(cor, annot=True, cmap=plt.cm.CMRmap_r)
plt.show()

In [16]:
df.groupby('CLIENT_ID')['PRICE_DISC'].sum()


## Recommendation to increase Revenue by adjusting discount

### Finding the Total revenue


In [17]:
df = pd.read_csv('../input/ibmvinit/IBM - IBM_CAO_data_challenge_DS_2021_.csv')

df.head()

In [18]:
Total_after_discount = df['PRICE_DISC'].sum()
Total_after_discount

# 562 million approx

### Now finding revenue by INDUSTRY as it is the most important feature

In [16]:
IT_rev = df.loc[df['INDUSTRY'] == 'IT', 'PRICE_DISC'].sum()
IT_rev

## 222 Mill

In [17]:
Finance_rev = df.loc[df['INDUSTRY'] == 'FINANCE', 'PRICE_DISC'].sum()
Finance_rev

## 101 Mill


In [18]:
REAL_rev = df.loc[df['INDUSTRY'] == 'REAL ESTATE', 'PRICE_DISC'].sum()
REAL_rev

## 90 Mill

In [19]:
RETAIL = df.loc[df['INDUSTRY'] == 'RETAIL', 'PRICE_DISC'].sum()
RETAIL

## 78 Mill

In [20]:
HEALTH_CARE = df.loc[df['INDUSTRY'] == 'HEALTH CARE', 'PRICE_DISC'].sum()
HEALTH_CARE

## 71 Mill

In [21]:

mylabels = ['IT', 'Finance', 'Real Estate','Retail','Health Care']
y = np.array([IT_rev,Finance_rev,REAL_rev,RETAIL,HEALTH_CARE])
plt.pie(y, labels = mylabels,autopct='%1.1f%%')
plt.title("Revenue distribution across industries")
plt.show() 

In [22]:

import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
x = ['IT', 'Finance', 'Real Estate','Retail','Health Care']
y = np.array([IT_rev,Finance_rev,REAL_rev,RETAIL,HEALTH_CARE])
ax.bar(x,y)
plt.title("Revenue distribution across industries")
plt.show()

#### As INDUSTRY is the most important feature to determine the discount, IT industry has the highest percentage share of total revenue that is 39.3% i.e. around 222 million. Reducing the discount percentage here will increase the revenue by a good amount

#### Let's calculate the mean of discount percentage across all the industries

In [64]:
IT_discount_mean = df.loc[df['INDUSTRY'] == 'IT', 'Discount_Percentage'].mean()
IT_discount_mean

In [65]:
Finance_discount_mean = df.loc[df['INDUSTRY'] == 'FINANCE', 'Discount_Percentage'].mean()
Finance_discount_mean

In [66]:
REAL_ESTATE_discount_mean = df.loc[df['INDUSTRY'] == 'REAL ESTATE', 'Discount_Percentage'].mean()
REAL_ESTATE_discount_mean


In [67]:
RETAIL_discount_mean = df.loc[df['INDUSTRY'] == 'RETAIL', 'Discount_Percentage'].mean()
RETAIL_discount_mean


In [68]:
Health_discount_mean = df.loc[df['INDUSTRY'] == 'HEALTH CARE', 'Discount_Percentage'].mean()
Health_discount_mean

In [76]:

import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
x = ['IT', 'Finance', 'Real Estate','Retail','Health Care']
y = np.array([IT_discount_mean,Finance_discount_mean,REAL_ESTATE_discount_mean,RETAIL_discount_mean,Health_discount_mean])
ax.bar(x,y)
plt.title("Discount percentage (Mean) distribution across industries")

plt.show()

In [78]:
IT_pric_mean = df.loc[df['INDUSTRY'] == 'IT', 'PRICE_ORIG'].mean()
IT_pric_mean
## average price before discount is 13124.59

In [79]:
IT_pric_dis_mean = df.loc[df['INDUSTRY'] == 'IT', 'PRICE_DISC'].mean()
IT_pric_dis_mean

## average price after discount 

In [ ]:
## suppose if we consider average discount percentage to be 35 % instead of 41 %, 
#The average price after discount will be 8530 compared to earlier 7387


#### Hence, IT Industry has the highest discount percentage of 41.65. As IT has both highest revenue and discount, reducing discount in this will increase the revenue.

#### The next important feature is Size, let's do revenue analysis on this.

In [80]:
df['SIZE'].value_counts()

In [81]:
zero_19 = df.loc[df['SIZE'] == '0-19', 'PRICE_DISC'].sum()
zero_19
## 237 mill

In [82]:
twenty_99 = df.loc[df['SIZE'] == '20-99', 'PRICE_DISC'].sum()
twenty_99

## 161 mill

In [83]:
Hundred_499 = df.loc[df['SIZE'] == '100-499', 'PRICE_DISC'].sum()
Hundred_499

## 127 mill

In [84]:
Five_Plus = df.loc[df['SIZE'] == '500+', 'PRICE_DISC'].sum()
Five_Plus

## 36 mill

In [96]:
mylabels = ['0-19','20-99','100-499','500+']
y = np.array([zero_19,twenty_99,Hundred_499,Five_Plus])
plt.pie(y, labels = mylabels,autopct='%1.1f%%')
plt.title("Revenue distribution across industries by sizes")
plt.show() 

#### MOST IMPORTANT CLIENTS

In [12]:
df_clients = df.groupby('CLIENT_ID')['PRICE_DISC'].sum()


In [13]:
sorted_series = df_clients.sort_values(ascending
                             = False)
sorted_series

In [91]:
#final_df = df.sort_values(by=['Discount_Percentage'], ascending=False)
#df['Discount_Percentage'].value_counts()

In [90]:
#Disc_85 = df.loc[df['Discount_Percentage'] > 50, 'PRICE_DISC'].sum()
#Disc_85

In [89]:
#Disc_85 = df[(df['Discount_Percentage'] > 50) & (df['INDUSTRY'] == 'IT')]
#Disc_85['PRICE_DISC'].sum()


## Product wise analysis of revenue

In [105]:
df['PROD_CAT'].value_counts()

In [92]:
SAAS = df.loc[df['PROD_CAT'] == 'SAAS', 'PRICE_DISC'].sum()
SAAS
## 4 mill

In [93]:
SW = df.loc[df['PROD_CAT'] == 'SW', 'PRICE_DISC'].sum()
SW
## 82 mill

In [94]:
HW = df.loc[df['PROD_CAT'] == 'HW', 'PRICE_DISC'].sum()
HW
## 462 mill

In [97]:
SS = df.loc[df['PROD_CAT'] == 'SS', 'PRICE_DISC'].sum()
SS
# 8 mill

In [100]:



import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
x = ['SAAS','SW','HW','SS']
y = np.array([SAAS,SW,HW,SS])
ax.bar(x,y)
plt.title("Revenue distribution across industries by products")

plt.show()

#### Clearly, HW is the the product that generates the maximum revenue i.e nearly 80%. Decreasing the discount percentage for this product will increase in revenue.

#### Let's check for the revenue generated by IT Industries from size 0-19 and for the product HW only.

In [102]:
Club = df[(df['PROD_CAT'] == 'HW') & (df['INDUSTRY'] == 'IT') & (df['SIZE'] == '0-19')]
Club['PRICE_DISC'].sum()

## 99 million